In [ ]:
# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# Install Tensorflow from the website: https://www.tensorflow.org/versions/r0.12/get_started/os_setup.html

# Installing Keras
# pip install --upgrade keras

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
% matplotlib inline

In [4]:
##### Part 1: data proprocessing #####
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X1= LabelEncoder()
X[:,1] = labelencoder_X1.fit_transform(X[:,1]) # replace country name with 0,1, and 2
X[:,2] = labelencoder_X1.fit_transform(X[:,2]) # replace gender name with 0 and 1
onehotencoder= OneHotEncoder(categorical_features=[1]) # create dummy variable only for 1st index(country[this has three categories]), other columns are numerical or 0/1
X= onehotencoder.fit_transform(X).toarray()
X= X[:,1:]  # drop the first columns to avoind dummy variable trap


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

Training the ANN with Stochastic Gradient Descent
* Step1: Randomly initialise the weights to small numbers close to zero
* Step2: Inupt the first observation of your dataset in the first input layer, each feature in one unput node
* Step3: Forward-Propagation: from left to right, the nerons are activated in a way that the impact of each neuron's activation is limited by the weights. Propagate the activations until getting the predicted result y.
* Step4: Compare the predicted result to the actual result. Measure the generated error.
* Step5: Back-Propagation: from right to left, the errors is back-propagated. Update the weights according to thow much they are responsible for the error. The learning rate decides by how much we update the weights.
* Step6: Repeat Steps one to five and update the weights after each observation (Reinforcement Learning) Or: Repeat Steps one to five but update the weights only after a batch of observatoins (Batch Learning)
* Step7: When the whole training set passed through the ANN, that makes an epoch. Redo more epochs

In [10]:
##### Part 2 #####
import keras
from keras.models import Sequential # a module required to initialise ANN
from keras.layers import Dense# a module required to build a layers of ANN

In [19]:
classifier = Sequential()   # initialise the ANN 
classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform', input_dim=11))# Add the input layer and the first hidden layer, 
classifier.add(Dense(units=6,  activation='relu',kernel_initializer='uniform')) # Nr of nodes in the hidden layer(`units`). --> (# nodes in the input layer(`input_dim`) + # nodes in the output layer)/2   
# Above is not formal way to choose `units` To be more precise, use parameter tuning to decide # nodes. (k-fold cross validation)[More in Ch.10]
classifier.add(Dense(units=1,activation='sigmoid', kernel_initializer='uniform')) # The last layer. If you are dealing with the model that has more than two categories as output (e.g. three), change to `units=3` and `activation='softmax'`    
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])   # compiling the ANN. `adam` for SGD (`loss= categorical_crossentropy` if you have more than three categories output)
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 4s 518us/step - loss: 0.4795 - acc: 0.7971
Epoch 2/100
8000/8000 [==============================] - 3s 372us/step - loss: 0.4046 - acc: 0.8116
Epoch 3/100
8000/8000 [==============================] - 3s 372us/step - loss: 0.3761 - acc: 0.8417
Epoch 4/100
8000/8000 [==============================] - 3s 379us/step - loss: 0.3593 - acc: 0.8566
Epoch 5/100
8000/8000 [==============================] - 3s 365us/step - loss: 0.3521 - acc: 0.8589
Epoch 6/100
8000/8000 [==============================] - 3s 355us/step - loss: 0.3490 - acc: 0.8589
Epoch 7/100
8000/8000 [==============================] - 3s 363us/step - loss: 0.3469 - acc: 0.8606
Epoch 8/100
8000/8000 [==============================] - 3s 362us/step - loss: 0.3458 - acc: 0.8605
Epoch 9/100
8000/8000 [==============================] - 3s 354us/step - loss: 0.3448 - acc: 0.8590
Epoch 10/100
8000/8000 [==============================] - 3s 356us/step - loss: 0.3437 - acc: 0.8601

In [22]:
##### Part 3: Making predictions and evaluationg the model
y_pred= classifier.predict(X_test)
y_pred=(y_pred>0.5)  # if y_pred is larger than 0.5, it returns True

In [24]:
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test, y_pred)
cm

array([[1516,   79],
       [ 202,  203]])

In [25]:
(1516+203)/len(y_pred)

0.8595

In [28]:
# to see which factor contributes how much (in the last epoch), 
classifier.get_weights()  # first, third and fifth elements(0,2,4) are coef matrix and second, fourth and sixth elements are biases (constants)

[array([[-0.0685491 ,  0.01031211, -0.14524947, -0.20919263, -0.29721656,
         -0.42603323],
        [-0.21672751,  0.14393991,  0.07026731,  0.07186568,  0.05595284,
         -0.05618538],
        [ 0.04218943,  0.0706072 ,  0.05942498, -0.14893547, -0.01005835,
          0.0018175 ],
        [ 0.01408772,  0.0607935 ,  0.04926321, -0.0887169 ,  0.00404141,
          0.21019055],
        [-0.64411891, -0.65037608,  0.43671867, -0.80878967, -0.16603443,
         -0.09495146],
        [-0.02797636, -0.03988308,  0.06470129,  0.08364291, -0.03687204,
         -0.00661068],
        [ 0.08363017,  0.07128832,  0.04780043,  0.0110533 ,  0.35861143,
          0.12057276],
        [ 0.14312515,  0.00901983,  0.22114325,  0.20619924,  1.01369047,
          0.77317202],
        [ 0.02082076,  0.17052664,  0.07429139, -0.15475869, -0.05799056,
         -0.15778933],
        [-0.37431344, -0.08920002,  0.90303278, -0.27137077,  0.0047048 ,
         -0.17934899],
        [-0.00885379,  0.15442